# Data Visualization

## Libraries

In [1]:
from preprocessing import load_and_prepare_model, temperature_scaled_softmax
import random
import os
import torch
from ultralytics.utils import ops
import numpy as np
from ultralytics.utils.tal import dist2bbox

# Taxon Class

In [2]:
class Taxon:
    def __init__(self, name, rank):
        self.name = name
        self.rank = rank
        self.parent = None
        self.children = []

    def set_parent(self, parent):
        """Sets the parent of this taxon and adds this taxon as a child to the parent."""
        self.parent = parent
        parent.add_child(self)

    def add_child(self, child):
        """Adds a child taxon to this taxon."""
        self.children.append(child)

    def __str__(self):
        """String representation of the taxon."""
        return f"{self.rank}: {self.name}"

class TaxonomyManager:
    def __init__(self):
        self.taxa = {}  # Stores all taxa regardless of rank

    def add_taxon(self, name, rank, parent_name=None):
        """Adds a taxon to the taxonomy."""
        taxon = Taxon(name, rank)
        self.taxa[name] = taxon
        if parent_name:
            parent = self.taxa.get(parent_name)
            if parent:
                taxon.set_parent(parent)

    def get_taxon(self, name):
        """Retrieves a taxon by its name."""
        return self.taxa.get(name)

    def display_taxonomy(self):
        """Displays the entire taxonomy in a hierarchical format."""
        # Find roots (taxa without parents)
        roots = [taxon for taxon in self.taxa.values() if taxon.parent is None]
        for root in roots:
            self._display_taxon(root, 0)

    def _display_taxon(self, taxon, indent):
        """Recursive helper function to display a taxon and its children."""
        print(' ' * indent + str(taxon))
        for child in taxon.children:
            self._display_taxon(child, indent + 4)

# Populate Taxonomy

In [3]:
taxonomy = TaxonomyManager()

# Populate Objects
object_names = ["marine life", "inanimate"]
for name in object_names:
    taxonomy.add_taxon(name, "Binary")

# Populate Binaries
binary_mapping = {
    "asteroidea": "marine life",
    "phaeophyceae": "marine life",
    "bivalia": "marine life",
    "myxini": "marine life",
    "artificial": "inanimate",
    "natural": "inanimate",
    "chlorophyta": "marine life",
    "monocots": "marine life"
}
for class_name, binary_name in binary_mapping.items():
    taxonomy.add_taxon(class_name, "Class", binary_name)

# Populate Classes
class_mapping = {
    "asterias": "asteroidea",
    "fucus": "phaeophyceae",
    "henrica": "asteroidea",
    "mya": "bivalia",
    "myxine": "myxini",
    "cylindrical": "artificial",
    "solid": "natural",
    "arboral": "natural",
    "saccharina": "phaeophyceae",
    "ulva": "chlorophyta",
    "urospora": "chlorophyta",
    "zostera": "monocots"
}
for genus_name, class_name in class_mapping.items():
    taxonomy.add_taxon(genus_name, "Genus", class_name)

# Populate Genera
genus_mapping = {
    "asterias rubens": "asterias",
    "asteroidea": "asterias",
    "fucus vesiculosus": "fucus",
    "henrica": "henrica",
    "mytilus edulis": "mya",
    "myxine glurinosa": "myxine",
    "pipe": "cylindrical",
    "rock": "solid",
    "saccharina latissima": "saccharina",
    "tree": "arboral",
    "ulva intestinalis": "ulva",
    "urospora": "urospora",
    "zostera marina": "zostera"
}
for species_name, genus_name in genus_mapping.items():
    taxonomy.add_taxon(species_name, "Species", genus_name)

# Model Hooks

# Extract Prediction from Yolo v8

In [4]:
# Load the model with random weights
random.seed(133745)
IMGDIR_PATH = "/mnt/RAID/datasets/label-studio/fjord/images/"
imgpath = IMGDIR_PATH + random.choice(os.listdir(IMGDIR_PATH))
model, hooks = load_and_prepare_model("runs/segment/Yolov8n-seg-train/weights/best.pt")

In [5]:
# Unpack hooks from load_and_prepare_model()
input_hook, detect, detect_hook, cv2_hooks, cv3_hooks = hooks

# Run inference
model(imgpath)

# Reverse engineer the outputs to find the logits
shape = detect_hook.input[0][0].shape  # BCHW
x = []
for i in range(detect.nl):
    x.append(torch.cat((cv2_hooks[i].output, cv3_hooks[i].output), 1))
x_cat = torch.cat([xi.view(shape[0], detect.no, -1) for xi in x], 2)
box, cls = x_cat.split((detect.reg_max * 4, detect.nc), 1)

# Batch size assumed to be 1
batch_idx = 0
dbox = dist2bbox(detect.dfl(box), detect.anchors.unsqueeze(0), xywh=True, dim=1) * detect.strides
cls = cls[batch_idx].sigmoid()

# Figure out the original img shape and model img shape to transform the boxes
img_shape = input_hook.input[0].shape[2:]
orig_img_shape = model.predictor.batch[1][batch_idx].shape[:2]

# Compute predictions
boxes = []
for i in range(dbox.shape[2]):  # Iterate over the third dimension (num_boxes)
    x0, y0, x1, y1 = dbox[0, :, i]  # Access all 4 coordinates for the i-th box
    x0, y0, x1, y1 = ops.scale_boxes(img_shape, np.array([x0.cpu(), y0.cpu(), x1.cpu(), y1.cpu()]), orig_img_shape)
    
    boxes.append({
        'image_id': imgpath,
        'bbox': [x0.item(), y0.item(), x1.item(), y1.item()],  # xyxy
        'bbox_xcycwh': [(x0.item() + x1.item())/2, (y0.item() + y1.item())/2, x1.item() - x0.item(), y1.item() - y0.item()],
        'logits': cls[:, i].tolist()
    })

# Prepare boxes for NMS
boxes_for_nms = torch.stack([
    torch.tensor([*b['bbox_xcycwh'], *b['logits']]) for b in boxes
], dim=1).unsqueeze(0)

nms_results = ops.non_max_suppression(boxes_for_nms, conf_thres=0.5, iou_thres=0.7)


WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /mnt/RAID/datasets/label-studio/fjord/images/frame013780.jpg: 640x576 1 ulva intestinalis, 1 urospora, 1 zostera marina, 347.2ms
Speed: 10.2ms preprocess, 347.2ms inference, 1924.5ms postprocess per image at shape (1, 3, 640, 576)


In [7]:

nms_results[0]

tensor([[115.4882, 566.1431, 217.5803, 104.9672,   0.9614,  12.0000],
        [115.1407, 566.0378, 217.6956, 105.0155,   0.9592,  12.0000],
        [115.3218, 566.2609, 217.5932, 104.8498,   0.9545,  12.0000],
        [115.3655, 565.9752, 217.3923, 105.2337,   0.9542,  12.0000],
        [114.9085, 565.8546, 219.3447, 105.7377,   0.9503,  12.0000],
        [115.2561, 566.1831, 217.0498, 104.6300,   0.9502,  12.0000],
        [115.4984, 566.3156, 217.3478, 104.6519,   0.9489,  12.0000],
        [115.2101, 566.1779, 218.1667, 104.8736,   0.9484,  12.0000],
        [115.4393, 566.3303, 217.4231, 104.6662,   0.9439,  12.0000],
        [115.4363, 566.1382, 217.5255, 105.2556,   0.9387,  12.0000],
        [448.7006, 515.2203, 158.2852,  68.5182,   0.9295,  11.0000],
        [448.9079, 515.2396, 157.7410,  68.7162,   0.9277,  11.0000],
        [448.5760, 515.3473, 158.1650,  68.8945,   0.9271,  11.0000],
        [448.6293, 515.1596, 156.5981,  68.7595,   0.9154,  11.0000],
        [448.5184, 5